<a href="https://colab.research.google.com/github/ruthetum/handson-ml-std/blob/master/ch11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 11 심층 신경망 훈련하기

In [1]:
# Settings
import sklearn
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

## 훈련 중 문제 상황
- 기울기 소실/폭주
- 훈련 데이터 부족
- 대규모 모델의 훈련 속도 저하

## 1. 기울기 소실/폭주 해결

문제 : 로지스틱 시그모이드 활성화 함수와 가중치 초기화 방법
- 로지스틱 활성화 함수에 의해 역전파가 진행될 때 최상위층에서부터 전달되는 그레디언트가 실제 아래층에는 도달하지 않게 됨

### 1) 글로럿 초기화 & He 초기화
- 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 그레디언트를 보장
- 그 대안으로 각 층의 연결 가중치를 무작위로 초기화

  **(1) 글로럿 초기화**
  ```
  keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")
  ```

  **(2) He 초기화**
  ```
  init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)
  ```

### 2) 수렴하지 않는 활성화 함수
- 활성화 함수를 잘못 선택하면 그레디언트의 소실이나 폭주로 이어짐
- ReLU는 계산이 빠르지만 완벽하지 않음 (ex. dying ReLU 문제)

  **(1) LeakyReLU**
  ```
  keras.layers.LeakyReLU() # alpha default : 0.3
  # or
  # keras.layers.LeakyReLU(alpha=0.2)
  ```
  **(2) PReLU**
  ```
  keras.layers.PReLU()
  ```
  **(3) RReLU**
  ```
  # 케라스 공식문서에 RReLU 구현은 아직 없음
  ```
  **(4) SELU**

  Günter Klambauer, Thomas Unterthiner, Andreas Mayr는 2017년 한 훌륭한 논문에서 SELU 활성화 함수를 소개했습니다. 훈련하는 동안 완전 연결 층만 쌓아서 신경망을 만들고 SELU 활성화 함수와 LeCun 초기화를 사용한다면 자기 정규화됩니다. 각 층의 출력이 평균과 표준편차를 보존하는 경향이 있습니다. 이는 그레이디언트 소실과 폭주 문제를 막아줍니다. 그 결과로 SELU 활성화 함수는 이런 종류의 네트워크(특히 아주 깊은 네트워크)에서 다른 활성화 함수보다 뛰어난 성능을 종종 냅니다. 하지만 SELU 활성화 함수의 자기 정규화 특징은 쉽게 깨집니다. ℓ1나 ℓ2 정규화, 드롭아웃, 맥스 노름, 스킵 연결이나 시퀀셜하지 않은 다른 토폴로지를 사용할 수 없습니다(즉 순환 신경망은 자기 정규화되지 않습니다). 하지만 실전에서 시퀀셜 CNN과 잘 동작합니다. 자기 정규화가 깨지면 SELU가 다른 활성화 함수보다 더 나은 성능을 내지 않을 것입니다.

  [출처] : https://github.com/rickiepark/handson-ml2/blob/master/11_training_deep_neural_networks.ipynb
  ```
  keras.layers.Dense(10, activation="selu",  kernel_initializer="lecun_normal")
  ```

  \+ Difference between ELU and SELU : z<0에서 곡선의 차이가 있음

#### cf. 심층 신경망의 은닉층에는 어떤 활성화 함수를 써야할까.
- 보통 **SELU > ELU > LeakyReLU(와 ReLU의 변형들) > ReLU > tanh > 로지스틱** 순으로
- 네트워크가 자기 정규화되지 못 하는 구조 : ELU > SELU 일 수도. 속도가 중요하다면 LeakyReLU도 괜찮
- 신경망이 과대적합됐다면 RReLU
- 훈련 세트가 아주 크다면 PReLU
- 속도가 가장 중요하다면 ReLU

### 3) Batch Normalization (배치 정규화)
 - ELU와 함께 He 초기화를 사용하면 훈련 초기 단계에서 그레디언트 소실/폭주 문제를 감소시킬 수 있지만, 훈련하는 동안 다시 발생할 수 있음 

In [2]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu" , kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [3]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [4]:
model.layers[1].updates

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


[]

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,  kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

### 4) Gradient Clipping (그레디언트 클리핑)
 - 역전파될 때 그레이언트의 임곗값을 설정하여 넘어서는 값은 제거
```
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)
```



## 2. 사전훈련된 층 재사용하기

### 1) 전이학습
- 비슷한 유형의 문제를 처리한 신경망을 찾아보고, 해당 신경망의 하위층을 재사용
```
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layer[:-1])
model_B_on_A = keras.models.load_model(1, activation="sigmoid"))
```
- 이 경우 model_B_on_A가 훈련할 때 A도 영향을 받음 (층을 공유하는 상황이기 때문)
- 영향을 원하지 않는 경우 복제해서 가중치를 복사
- 먼저 원래 모델 복제 후 가중치 저장
``` 
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
```
- 새로운 층에게 가중치를 학습
```
for layer in model_B_on_A.layer[:-1]:
    layer.trainable = False # 기존 가중치를 변형하지 않기 위해 false
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"])
```
- 이후 다시 컴파일
```
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
optimizer=keras.optimizers.SGD(lr=1e-4) # default 1e-2
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))
```


### 2) 비지도 사전훈련
- 레이블된 훈련 데이터가 많지 않은 경우, 비슷한 작업에 대해 훈련된 모델을 찾을 수 없는 경우
- 오토인코더, GNA

## 3. 고속 옵티마이저

- 경사 하강법 대신 더 빠른 옵티마이저를 사용

(1) 모멘텀 최적화

(2) 네스테로프 가속 경사

(3) AdaGrad

(4) RMSProp

(5) Adam, Nadam

### (1) 모멘텀 최적화
```
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)
```
### (2) 네스테로프 가속 경사
```
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)
```
### (3) AdaGrad
```
optimizer = keras.optimizers.Adagrad(lr=0.001)
# 간단한 2차 방정식 문제에 대해서는 잘 작동하지만 신경망을 훈련할 때 종종 너무 일찍 멈춤
# 따라서 심층 신경망에서는 사용하지 않는 게 좋음
```
### (4) RMSProp
```
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
```
### (5) Adam
```
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
```
### (5)-(1) AdaMax
```
optimizer = keras.optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999)
# Adam이 잘 작동하지 않을 때 사용해볼만 함
```
### (5)-(2) Nadam
```
# Adam + 네스테로프
optimizer = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)
```
### (6) 학습률 스케줄링
- 거듭제곱 기반 스케줄링
- 지수 기반 스케줄링
- 구간별 고정 스케줄링
- 성능 기반 스케줄링
- 1 사이클 스케줄링


## 4. 규제를 사용해 과대적합 피하기

### 1) L1, L2 규제
- L1 : (많은 가중치가 0인) 희소 모델을 만들기 위해 사용
```
kernel_regularizer=keras.regularizers.l1(0.01)
```
- L2 : 신경망의 연결 가중치를 제한하기 위해 사용
```
kernel_regularizer=keras.regularizers.l2(0.01)
```
- Both L1 and L2
```
kernel_regularizer=keras.regularizers.l1_l2(0.01)
```

### 2) Dropout (드롭아웃)
```
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
```

### 3) 몬테 카를로 드롭아웃
```
y_probas = np.stack([model(X_test_scaled, training=True) for sample in range(100)])
y_proba = y_probas.mean(axis=0)
np.round(model.predict(X_test_scaled[:1]), 2)
```

### 4) 맥스-노름 규제
```
ayer = keras.layers.Dense(100, activation="selu",kernel_initializer="lecun_normal", kernel_constraint=keras.constraints.max_norm(1.))
```


